In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from utils.constants import Conv1d_ft, Conv1d_st
from utils.NeuralNetwork import Conv1DNet

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

In [5]:
train_data = np.load("Train_train.npz")
test_data = np.load("Test_Exp_test.npz")
Xtr, ytr = train_data['arr_0'], train_data['arr_1']
Xte, yte = test_data['arr_0'] , test_data['arr_1']

torch.tensor(Xtr), torch.tensor(ytr)

train_dataset = TensorDataset(torch.tensor(Xtr), torch.tensor(ytr))
test_dataset = TensorDataset(torch.tensor(Xte), torch.tensor(yte))

train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, drop_last=True)

In [7]:
fc1_input_dim = Conv1d_ft[1] * (Conv1d_ft[1] - Conv1d_st[0] + 1) #CC
#net = Conv1DNet(fc1_input_dim).cuda()
net = Conv1DNet(fc1_input_dim)
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [9]:
loss_curve = []
acc_curve = []

for _ in range(20):

    for i, (xbatch, ybatch) in enumerate(train_loader):
        xbatch, ybatch = xbatch.float(), ybatch.long()

        optimizer.zero_grad()

        ypred = net(xbatch)
        loss = criterion(ypred, ybatch)

        loss.backward()

        optimizer.step()
        
        # if i % 100 == 0:
            
            # print(f'Trainning Loss {loss.item()}')
        
    with torch.no_grad():
        ypred_list = []
        ybatch_list = []
        loss_list = []
        for i, (xbatch, ybatch) in enumerate(test_loader):
            xbatch, ybatch = xbatch.float(), ybatch.float()
            
            ypred = net(xbatch)
            loss = criterion(ypred, ybatch)
            
            ybatch_list.append(ybatch)
            ypred_list.append(ypred.max(dim=1)[1])
            loss_list.append(loss)
            
        loss_test_all = torch.hstack(loss_list).cpu().numpy()
        ytest_pred = torch.hstack(ypred_list).cpu().numpy()
        ytest_real = torch.hstack(ybatch_list).cpu().numpy()
        
        acc = accuracy_score(ytest_pred, ytest_real)
        
        print(f'After testing, the loss is {loss_test_all.mean()}, the accuracy is {acc}')

torch.Size([64, 128, 126])
torch.Size([64, 64, 124])
torch.Size([64, 64, 62])
torch.Size([64, 150])


IndexError: Target 14 is out of bounds.